In [18]:
%%capture
!pip install datasets
!pip install transformers
!pip install torchaudio
!pip install librosa
!pip install jiwer
!pip install pandas
!pip install scikit-learn
!pip install numpy

In [6]:
import pandas as pd

colnames = ['fpath', 'userid', 'text']
df = pd.read_csv('./content/asr_nepali/utt_spk_text.tsv', sep='\t', header=None, names=colnames)
# Folder structure for the files
df['path'] = './content/asr_nepali/data/' + df['fpath'].str[:2]
# Add file link to the path
df['path'] = df['path'] + "/" + df['fpath'] + '.flac'
# Drop Duplicates
df = df.drop_duplicates(subset=['text'])


FileNotFoundError: [Errno 2] No such file or directory: './content/asr_nepali/utt_spk_text.tsv'

In [ ]:
df

In [ ]:
from sklearn.model_selection import train_test_split

x_train, x_test = train_test_split(df, train_size=0.8)

In [ ]:
x_train

In [ ]:
from datasets import Dataset


# Load Dataset from dataframe
dataset_test = Dataset.from_pandas(x_test)
dataset_train = Dataset.from_pandas(x_train)

dataset_test = dataset_test.remove_columns(['fpath', 'userid', '__index_level_0__'])
dataset_train = dataset_train.remove_columns(['fpath', 'userid', '__index_level_0__'])

In [ ]:
import re

# Remove characters to be ignored using regex
chars_to_ignore_regex = '[\,\?\.\!\-\;\:\"\“\%\‘\”\�]'


# Function to remove special characters/characters to ignore
def remove_special_characters(batch):
    batch['text'] = re.sub(chars_to_ignore_regex, '', batch['text']).lower() + " "


dataset_test = dataset_test.map(remove_special_characters)
dataset_train = dataset_train.map(remove_special_characters)

In [ ]:
# Function to extract all the individual characters in the transcripts
def extract_all_chars(batch):
    all_text = " ".join(batch["text"])
    vocab = list(set(all_text))
    return {"vocab": [vocab], "all_text": [all_text]}


# Mapping the datasets to the extract_all_chars function and generate the vocabulary
vocab_test = dataset_test.map(extract_all_chars, batched=True, batch_size=-1, keep_in_memory=True,
                              remove_columns=dataset_test.column_names)
vocab_train = dataset_train.map(extract_all_chars, batched=True, batch_size=-1, keep_in_memory=True,
                                remove_columns=dataset_train.column_names)
# Generate the complete vocabulary from both test and train vocabularies
vocab_list = list(set(vocab_train['vocab'][0]) | set(vocab_test['vocab'][0]))

In [ ]:
import json
# Generate the vocabulary dictionary
vocab_dict = {v: k for k, v in enumerate(vocab_list)}

# Remove unwanted characters from the vocabulary
try:
    del vocab_dict['\\']
    del vocab_dict['a']
    del vocab_dict['b']
    del vocab_dict['c']
    del vocab_dict['e']
    del vocab_dict['f']
    del vocab_dict['k']
    del vocab_dict['o']
    del vocab_dict['\xa0']
    del vocab_dict['\u200c']
    del vocab_dict['\u200d']
    del vocab_dict['\u200e']
    del vocab_dict['\u200f']
except:
    pass

# Replace the space " " key to a visible character "|"
vocab_dict["|"] = vocab_dict[" "]
del vocab_dict[" "]
# Create keys for unknown and paddings
vocab_dict["[UNK]"] = len(vocab_dict)
vocab_dict["[PAD]"] = len(vocab_dict)

with open('vocab.json', 'w') as vocab_file:
    json.dump(vocab_dict, vocab_file)

In [8]:
from transformers import Wav2Vec2CTCTokenizer

# Create a Wav2Vec2CTCTokenizer object with the generated vocabulary json with the required tokens
tokenizer = Wav2Vec2CTCTokenizer("./vocab.json", unk_token="[UNK]", pad_token="[PAD]", word_delimiter_token="|")

In [9]:
from transformers import Wav2Vec2FeatureExtractor

# Create a Wav2Vec2FeatureExtractor object for the audio
feature_extractor = Wav2Vec2FeatureExtractor(feature_size=1, sampling_rate=16000, padding_value=0.0, do_normalize=True,
                                             return_attention_mask=True)

In [10]:
from transformers import Wav2Vec2Processor
# Create Wav2Vec2Processor object using the created feature_extractor and tokenizer
processor = Wav2Vec2Processor(feature_extractor=feature_extractor, tokenizer=tokenizer)

In [ ]:
dataset_test

In [15]:
import torchaudio
import random
import gc
# Convert audio to numpy array for processing
def speech_file_to_array_fn(batch):
    batch = Dataset.from_dict(batch)
    batch = batch.map(gen_speech_arr)
    return batch.to_dict()

def gen_speech_arr(file):
    speech_array, sampling_rate = torchaudio.load(file["path"])
    file["speech"] = speech_array[0].numpy()
    file["sampling_rate"] = sampling_rate
    file["target_text"] = file["text"]
    if random.random() < 0.25:
        gc.collect()
    return file


d:\course_tbc\4th year\pp\fyp\project\venv\lib\site-packages\torchaudio\extension\extension.py:13: UserWarning: torchaudio C++ extension is not available.
  warnings.warn('torchaudio C++ extension is not available.')


In [16]:
dataset_test = dataset_test.map(speech_file_to_array_fn, remove_columns=dataset_test.column_names,batched=True)

In [17]:
dataset_train = dataset_train.map(speech_file_to_array_fn, remove_columns=dataset_train.column_names,batched=True)

In [21]:
dataset_train.save_to_disk("./processed_data/train")

In [ ]:
dataset_test.save_to_disk("./processed_data/test")

In [11]:
from datasets import load_from_disk
dataset_train = load_from_disk("./processed_data/train")

In [12]:
dataset_test = load_from_disk("./processed_data/test")

In [20]:
import librosa

In [21]:
def resample(batch):
    batch["speech"] = librosa.resample(np.asarray(batch["speech"]), 48_000, 16_000)
    batch["sampling_rate"] = 16_000
    return batch

In [22]:
dataset_train = dataset_train.map(resample, num_proc=4)

NameError: name 'librosa' is not defined

In [23]:
def prepare_dataset(batch):
    batch["input_values"] = processor(batch["speech"], sampling_rate=batch["sampling_rate"][0]).input_values

    with processor.as_target_processor():
        batch["labels"] = processor(batch["target_text"]).input_ids
    return batch

In [24]:

dataset_train = dataset_train.map(prepare_dataset, remove_columns=dataset_train.column_names, batch_size=8, num_proc=4,
                                  batched=True)


NameError: name 'processor' is not defined